In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
import torch

In [2]:
def zero(text):
    l=[0]*len(text)
    return(l)

In [3]:
def get_user(l):
    ll=[]
    for x in l:
        ll.append(x['user'])
    return(list(set((ll))))

In [4]:
def index_by_user(l):
    dico = {}
    for user in get_user(l):
        dico[user] = []
        for x in l:
            if x['user'] == user:
                dico[user].append(x['start_offset'])
                dico[user].append(x['end_offset'])
    return(dico)

In [5]:
def delete_not_read(df):
    data = df.copy()
    columns = list(data.columns)
    columns.remove('index')
    columns.remove('text')
    l = []
    for user in columns:
        for i in range(len(data)):
                if (data[user][i][0] == 1) and (i not in l):
                    l.append(i)
    
    return(data.drop(l, axis = 0))

In [6]:
def max_len(df):
    max_len = 0

    for sent in df['text']:

        input_ids = tokenizer.encode(sent, add_special_tokens=True)

        # Update the maximum sentence length.
        max_len = max(max_len, len(input_ids))

    return(max_len)

In [7]:
def tokenize_and_mask(text, maxlen):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = maxlen,           # Pad & truncate all sentences.
                        padding = 'longest',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    return(encoded_dict['input_ids'], encoded_dict['attention_mask'])

In [8]:
def clean_doccano(df):
    data = pd.DataFrame()
    data['text'] = df['text'].apply(lambda x: (" ".join((str(x).split()))).replace('\\',""))
    data['index'] = df['annotations'].apply(index_by_user)
    print("Création de la liste binaire...")
    for x in list(set(df['annotations'].apply(get_user).sum())):
        data[f'{x}'] = data['text'].apply(zero)
    for i in range(len(data)):    
        dico = data['index'][i]
        for user in dico.keys():
            l = data[f'{user}'].values[i]
            index = dico[user]
            for j in range(0,len(index),2):
                l[index[j]:index[j+1]] = [1]*(index[j+1]-index[j])
            data.loc[i, f'{user}'] = l
    print("... suppression des texte non annotés ...")
    data = delete_not_read(data)
    maxlen = max_len(data)
    print("...tokenization du texte")
    data['token'] = data['text'].apply(lambda x: tokenize_and_mask(x, maxlen)[0])
    data['mask'] = data['text'].apply(lambda x: tokenize_and_mask(x, maxlen)[1])
    bdd = data.drop(['text', 'index'], axis = 1)
    i=1
    for user in list(bdd.columns)[:-2]:
        bdd.rename( columns = {user : f'Annotateur {i}'}, inplace = True)
        i+=1
    return(bdd)